# 🧠 IA + Oracle: Hola mundo con Gemini API Key

Este notebook muestra cómo conectar una base de datos Oracle y generar consultas SQL desde lenguaje natural usando el modelo Gemini de Google vía su librería oficial `google.generativeai`.

🔑 Solo necesitas una clave API (`GOOGLE_API_KEY`).  
❌ No se requiere `project_id`, `region` ni cuenta de servicio.

Ideal para prototipos rápidos y prácticas educativas.

## 🔍 Comparativa: Vertex AI vs Gemini API Key

| Característica              | Vertex AI (AISuite)         | Gemini API Key (`google.generativeai`) |
|----------------------------|-----------------------------|----------------------------------------|
| Requiere `project_id`      | ✅ Sí                        | ❌ No                                   |
| Requiere `region`          | ✅ Sí                        | ❌ No                                   |
| Requiere cuenta de servicio| ✅ Sí                        | ❌ No                                   |
| Ideal para producción      | ✅                           | 🔸 Solo para prototipos y pruebas       |
| Simplicidad de uso         | 🔸 Más compleja              | ✅ Muy sencilla                         |

In [1]:
# 🔐 Configurar Gemini con API Key
from dotenv import load_dotenv
import os
import google.generativeai as genai

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")

In [5]:
# 🔗 Conexión a Oracle
import oracledb

def get_connection():
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    dsn = os.getenv("DB_DSN")
    wallet_location = os.getenv("WALLET_LOCATION")
    wallet_password = os.getenv("WALLET_PASSWORD")
    
    missing = [name for name, val in (("DB_USER", user), ("DB_PASSWORD", password), ("DB_DSN", dsn)) if not val]
    if missing:
        raise RuntimeError(f"Missing environment variables: {', '.join(missing)}")
    
    return oracledb.connect(
        user=user,
        password=password,
        dsn=dsn,
        wallet_location=wallet_location,
        wallet_password=wallet_password
    )

connection = get_connection()

cursor = connection.cursor()

ORACLE_USER = os.getenv("DB_USER")

cursor.execute("SELECT USER FROM dual")
print("Conectado como:", cursor.fetchone()[0])

Conectado como: MALACKATHON


In [6]:
# 🗂️ Descubrimiento del esquema
cursor.execute(f"""
    SELECT table_name, column_name
    FROM all_tab_columns
    WHERE owner = UPPER('{ORACLE_USER}')
    ORDER BY table_name, column_id
""")

esquema = {}
for table, column in cursor:
    esquema.setdefault(table, []).append(column)

print("Esquema de la base de datos:")
for tabla, columnas in esquema.items():
    print(f"{tabla}({', '.join(columnas)})")

Esquema de la base de datos:
DBTOOLS$EXECUTION_HISTORY(ID, HASH, CREATED_BY, CREATED_ON, UPDATED_BY, UPDATED_ON, STATEMENT, TIMES)
ENFERMEDADESMENTALESDIAGNOSTICO(Comunidad Autónoma, NOMBRE, FECHA_DE_NACIMIENTO, SEXO, CCAA_RESIDENCIA, FECHA_DE_INGRESO, CIRCUNSTANCIA_DE_CONTACTO, FECHA_DE_FIN_CONTACTO, TIPO_ALTA, Estancia Días, Diagnóstico Principal, Categoría, Diagnóstico 2, Diagnóstico 3, Diagnóstico 4, Diagnóstico 5, Diagnóstico 6, Diagnóstico 7, Diagnóstico 8, Diagnóstico 9, Diagnóstico 10, Diagnóstico 11, Diagnóstico 12, Diagnóstico 13, Diagnóstico 14, Fecha de Intervención, PROCEDIMIENTO_1, PROCEDIMIENTO_2, PROCEDIMIENTO_3, PROCEDIMIENTO_4, PROCEDIMIENTO_5, PROCEDIMIENTO_6, PROCEDIMIENTO_7, PROCEDIMIENTO_8, PROCEDIMIENTO_9, PROCEDIMIENTO_10, PROCEDIMIENTO_11, PROCEDIMIENTO_12, PROCEDIMIENTO_13, PROCEDIMIENTO_14, PROCEDIMIENTO_15, PROCEDIMIENTO_16, PROCEDIMIENTO_17, PROCEDIMIENTO_18, PROCEDIMIENTO_19, PROCEDIMIENTO_20, GDR_AP, CDM_AP, TIPO_GDR_AP, Valor Peso Español, GRD_APR, CDM_A

In [9]:
# 🧠 Pregunta en lenguaje natural
pregunta = "¿Cuántas filas tiene la tabla de enfermos mentales?"

In [10]:
# 🤖 Generación de SQL con Gemini
esquema_texto = "\n".join(
    f"{tabla}({', '.join(columnas)})" for tabla, columnas in esquema.items()
)

prompt_sql = f"""
Eres un asistente experto en SQL para Oracle. Genera solo la consulta SQL compatible con Oracle.
Usa este esquema de base de datos:
{esquema_texto}

Pregunta del usuario:
{pregunta}
"""

raw_sql = model.generate_content(prompt_sql)
sql_generado = raw_sql.text.strip().strip("```sql").strip("```")
sql_generado = sql_generado.replace(";", "")  # Elimina el punto y coma si está presente
#eliminar saltos de línea y tabulaciones pues dan error en Oracle
sql_generado = sql_generado.replace("\n", " ").replace("\t", " ")


print("Consulta generada por IA:")
print(sql_generado)

Consulta generada por IA:
 SELECT   COUNT(*) FROM ENFERMEDADESMENTALESDIAGNOSTICO 


In [11]:
# 🧪 Ejecutar SQL y mostrar resultados
cursor.execute(sql_generado)
resultados = cursor.fetchall()
columnas = [col[0] for col in cursor.description]

import pandas as pd
df = pd.DataFrame(resultados, columns=columnas)
texto_resultado = df.to_markdown(index=False)
df.head()

,COUNT(*)
0,21210


In [12]:
# 🧠 Interpretación con Gemini
prompt_explicacion = f"""
Eres un experto en análisis de datos. Resume e interpreta los resultados de una consulta SQL.
La pregunta original fue: {pregunta}
Los resultados fueron:

{texto_resultado}
"""

response = model.generate_content(prompt_explicacion)
print("🧠 Respuesta interpretada por IA:")
print(response.text.strip())

🧠 Respuesta interpretada por IA:
¡Excelente! Como experto en análisis de datos, procedo a resumir e interpretar los resultados de esta consulta SQL.

---

### Resumen e Interpretación de Resultados de Consulta SQL

**1. Pregunta Original de la Consulta:**
La pregunta formulada a la base de datos fue: "¿Cuántas filas tiene la tabla de enfermos mentales?"
El objetivo era obtener el conteo total de registros en una tabla específica, presumiblemente identificada como `tabla de enfermos mentales`.

**2. Resultados Obtenidos:**
La consulta ejecutada (`COUNT(*)`) devolvió el siguiente resultado:
`21210`

**3. Interpretación de los Resultados:**

*   **Conteo Total de Registros:** El número `21.210` representa el **número exacto de filas** presentes en la tabla que fue consultada.
*   **Volumen de Datos:** Esto significa que, en el momento de la ejecución de la consulta, existen **21.210 registros individuales** en la tabla de "enfermos mentales". Asumiendo que cada fila representa un caso úni

## 🧠 Reflexión

- ¿Qué ventajas tiene usar Gemini con API Key?
- ¿Qué limitaciones presenta frente a Vertex AI?
- ¿Cómo podrías extender este flujo para validar o visualizar los resultados?

Este notebook es ideal para prácticas rápidas, pero recuerda que para entornos profesionales, Vertex AI ofrece mayor control y seguridad.